In [4]:
import pandas as pd
import numpy as np

def parseSpreadsheet(filename, sheetStart, sheetEnd, verbose=False):
    sheetNum = sheetStart
    while(sheetNum <= sheetEnd):
        df = pd.read_excel(filename, sheet_name=sheetNum)
        df = df.ffill(axis=0) # fill empty cells

        # assign spreadsheet columns to variable names
        partFamily = df.columns.values[0]
        manufacturerPart = df.columns.values[1]
        LCSC_Code = df.columns.values[4]
        purchaseQuantity = df.columns.values[5]
        df[df.columns[5]] = df[df.columns[5]].astype("int64") # convert purchase quantity to int

        # extract list of family names
        df_groups = df.groupby([partFamily], observed=False).head(1)
        families = np.array(df_groups[partFamily])

        # loop through each family, extracting manufacturer part, LCSC code, and purchase quantity
        df_perF = []
        for f in families:
            df_F = df[df[partFamily] == f].get([partFamily, manufacturerPart, LCSC_Code, purchaseQuantity])
            df_perF.append(df_F)
        if(verbose):
            for F in df_perF:
                print(F)
                print('------------------------------------------------')

        sheetNum += 1
    
    return df_perF

########################

filename = r'LCSC BOM H26.xlsx'
df_perF = parseSpreadsheet(filename, 1, 1)
# print(type(df_perF[0]))

df = df_perF[0]
print(df)
# print(type(df))
# print(df[df.columns[1]].astype(str).tolist())

# # test looping through part names
# for df in df_perF:
#     # print(df)
#     # print(type(df))
#     partNumbers = df[df.columns[1]].astype(str).tolist()
#     print(partNumbers)

#     for partNum in partNumbers:
#         print(type(partNum), partNum)

# # extract purchase quantity
# for df in df_perF:
#     family, quantity = df.iloc[0, [0, 3]]
#     print(family, quantity)

  Part Family (Customer PN)  Manufacturer Part Number  Package  \
0                      5003                       NaN      NaN   

   Purchase Quantity  
0                  4  


In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time


########################


# manual settings for development/testing
# purchaseQuantity = 7

# part_numbers = ["C108516"]
# part_numbers = ["C2053506"]
# part_numbers = ["C404027"]
# part_numbers = ["C108516", "C2053506"]
# part_numbers = ["C108516", "C2053506", "C1337223", "C2055188", "C81523", "C2053703", "C2970020"] # MCU
# part_numbers = ["C404027", "C2937042", "C30197532"] # LDO

# df = df_perF[0]
# partNumbers = df[df.columns[1]].astype(str).tolist()


########################

waitDuration = 1 # seconds. Don't DDOS LCSC

service = Service(executable_path=r'./driver/geckodriver.exe', log_output=r"./driver/gecko_log.txt")
driver = webdriver.Firefox(service=service)

df_exportList = pd.DataFrame(columns=['Part Family', 'LCSC Code', 'Stock Status', 'Stock Quantity', 'Unit Price', 'Purchase Quantity']) # df of cheapest parts

for df in df_perF:
    partFamily, purchaseQuantity = df.iloc[0, [0, 3]]
    partNumbers = df[df.columns[2]].astype(str).tolist()
    print(partNumbers)
    df = pd.DataFrame(columns=['Part Family', 'LCSC Code', 'Stock Status', 'Stock Quantity', 'Unit Price', 'Purchase Quantity']) # init output df for this family

    # loop through each part in family and web-scrape
    for partNum in partNumbers:
        print(partNum)
        if(partNum == "--"): # no LCSC code, so skip iteration
            continue
        # navigate to product page
        productUrl = f"https://www.lcsc.com/search?q={partNum}&s_z=n_{partNum}"
        driver.get(productUrl)
        time.sleep(waitDuration) # Don't DDOS LCSC

        # get LCSC stock status and available quantity
        stockStatusElementClass = driver.find_element(By.CLASS_NAME, "detailRightPanelWrap")
        # print(stockStatusElementClass.get_property("innerHTML"))
        stockStatusElement = stockStatusElementClass.find_element(By.XPATH, "//span[contains(text(), 'Stock')]")
        # print(stockStatusElement.get_property("innerHTML"))
        stockStatus = "In-Stock" if "In-Stock" in stockStatusElement.text else "Out of Stock"
        stockQuantity = stockStatusElement.text.split(":")[1].strip() if stockStatus == "In-Stock" else "0"
        stockQuantity = int(stockQuantity.strip().replace(",", ""))
        # print("Stock Status:\t" + stockStatus)
        # print("Stock Quantity:\t" + str(stockQuantity))
        if(stockStatus == "Out of Stock" or stockQuantity < purchaseQuantity): # skip iteration if out of stock or not enough stock
            continue
        # input desired purchase quantity into price calculator
        priceCalculatorElement = driver.find_element(By.CLASS_NAME, "quantityReelWrap")
        priceCalculatorInputElement = priceCalculatorElement.find_element(By.CSS_SELECTOR, "input[maxlength='9']")
        priceCalculatorInputElement.clear()  # Clear any existing text
        priceCalculatorInputElement.send_keys(str(purchaseQuantity))
        time.sleep(0.1) # Wait for input to register
        priceCalculatorInputElement.send_keys(Keys.RETURN)

        # get LCSC unit price for the given quantity
        purchaseElement = driver.find_element(By.CLASS_NAME, 'reelResultWrap')
        dataRow = purchaseElement.find_element(By.CSS_SELECTOR, '.row.mt6')
        dataColumns = dataRow.find_elements(By.CSS_SELECTOR, '.col.col-3')
        roundPurchaseQuantity = int(dataColumns[1].find_element(By.CLASS_NAME, 'major2--text').text.strip().replace(",", ""))
        # print("Purchase Quantity:\t" + str(roundPurchaseQuantity))
        unitPriceElement = dataColumns[2].find_element(By.XPATH, ".//div[contains(text(), '$')]")
        unitPrice = float(unitPriceElement.text.replace("$", "").strip())
        # print("Unit Price:\t" + str(unitPrice))

        # print("") # new line
        time.sleep(waitDuration) # Don't DDOS LCSC

        # export to df
        df.loc[len(df)] = [partFamily, partNum, stockStatus, stockQuantity, unitPrice, roundPurchaseQuantity]

    # select cheapest part in family (only if there are available parts)
    if not df.empty:
        minPart = df.loc[df[df.columns[3]].idxmin()].to_frame().T
        df_exportList = pd.concat([df_exportList, minPart], ignore_index = True)
    else:
        print(f"Warning: No available parts found for family '{partFamily}' with sufficient stock")

    # export dataframe for LCSC BOM list
    print(df)
    print(df_exportList)

########################



driver.quit() # close the browser

# export dataframe for LCSC BOM list
print(df_exportList)
# df.to_excel("output.xlsx", index=False)
# df.to_csv("output.csv", index=False)

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time

def scrapePartNumbers(df_perF, verbose=False):
    waitDuration = 1 # seconds. Don't DDOS LCSC

    df_exportList = pd.DataFrame(columns=['Part Family', 'LCSC Code', 'Stock Status', 'Stock Quantity', 'Unit Price', 'Purchase Quantity', 'Rounded Purchase Quantity']) # df of cheapest parts

    for df in df_perF:
        partFamily, purchaseQuantity = df.iloc[0, [0, 3]]
        partNumbers = df[df.columns[2]].astype(str).tolist()
        if(verbose):
            print(partNumbers)
        df = pd.DataFrame(columns=['Part Family', 'LCSC Code', 'Stock Status', 'Stock Quantity', 'Unit Price', 'Purchase Quantity', 'Rounded Purchase Quantity']) # init output df for this family
        
        # loop through each part in family and web-scrape
        for partNum in partNumbers:
            if(verbose):
                print(partNum)
            if(partNum == "--"): # no LCSC code, so skip iteration
                continue
            # navigate to product page
            productUrl = f"https://www.lcsc.com/search?q={partNum}&s_z=n_{partNum}"
            driver.get(productUrl)
            time.sleep(waitDuration) # Don't DDOS LCSC

            # get LCSC stock status and available quantity
            stockStatusElementClass = driver.find_element(By.CLASS_NAME, "detailRightPanelWrap")
            # print(stockStatusElementClass.get_property("innerHTML"))
            stockStatusElement = stockStatusElementClass.find_element(By.XPATH, "//span[contains(text(), 'Stock')]")
            # print(stockStatusElement.get_property("innerHTML"))
            stockStatus = "In-Stock" if "In-Stock" in stockStatusElement.text else "Out of Stock"
            stockQuantity = stockStatusElement.text.split(":")[1].strip() if stockStatus == "In-Stock" else "0"
            stockQuantity = int(stockQuantity.strip().replace(",", ""))
            # print("Stock Status:\t" + stockStatus)
            # print("Stock Quantity:\t" + str(stockQuantity))
            if(stockStatus == "Out of Stock" or stockQuantity < purchaseQuantity): # skip iteration if out of stock or not enough stock
                continue
            # input desired purchase quantity into price calculator
            priceCalculatorElement = driver.find_element(By.CLASS_NAME, "quantityReelWrap")
            priceCalculatorInputElement = priceCalculatorElement.find_element(By.CSS_SELECTOR, "input[maxlength='9']")
            priceCalculatorInputElement.clear()  # Clear any existing text
            priceCalculatorInputElement.send_keys(str(purchaseQuantity))
            time.sleep(0.1) # Wait for input to register
            priceCalculatorInputElement.send_keys(Keys.RETURN)

            # get LCSC unit price for the given quantity
            purchaseElement = driver.find_element(By.CLASS_NAME, 'reelResultWrap')
            dataRow = purchaseElement.find_element(By.CSS_SELECTOR, '.row.mt6')
            dataColumns = dataRow.find_elements(By.CSS_SELECTOR, '.col.col-3')
            roundPurchaseQuantity = int(dataColumns[1].find_element(By.CLASS_NAME, 'major2--text').text.strip().replace(",", ""))
            # print("Purchase Quantity:\t" + str(roundPurchaseQuantity))
            unitPriceElement = dataColumns[2].find_element(By.XPATH, ".//div[contains(text(), '$')]")
            unitPrice = float(unitPriceElement.text.replace("$", "").strip())
            # print("Unit Price:\t" + str(unitPrice))

            # print("") # new line
            time.sleep(waitDuration) # Don't DDOS LCSC

            # export to df
            df.loc[len(df)] = [partFamily, partNum, stockStatus, stockQuantity, unitPrice, purchaseQuantity, roundPurchaseQuantity]

        # select cheapest part in family (only if there are available parts)
        if not df.empty:
            minPart = df.loc[df[df.columns[3]].idxmin()].to_frame().T
            df_exportList = pd.concat([df_exportList, minPart], ignore_index = True)
        else:
            if(verbose):
                print(f"Warning: No available parts found for family '{partFamily}' with sufficient stock")

        # export dataframe for LCSC BOM list
        if(verbose):
            print("All parts:\n", df, "\n")
            print("Cheapest:\n", df_exportList, "\n")

    return df_exportList

########################

service = Service(executable_path=r'./driver/geckodriver.exe', log_output=r"./driver/gecko_log.txt")
driver = webdriver.Firefox(service=service)


df_exportList = scrapePartNumbers(df_perF, verbose=True)

driver.quit() # close the browser

# export dataframe for LCSC BOM list
print(df_exportList)
# df.to_excel("output.xlsx", index=False)
df_exportList.to_csv("output.csv", index=False)